# 进程与线程

### 并发与并行

#### 并发

并发是指在单个 CPU 上处理多个任务，任务在一段时间内交替执行。每个任务运行一小段时间后，CPU 切换到下一个任务，从而看起来像是同时运行。

In [ ]:
import time

def task1():
    print("任务 1 开始")
    time.sleep(1)
    print("任务 1 完成")

def task2():
    print("任务 2 开始")
    time.sleep(1)
    print("任务 2 完成")

task1()
task2()

这段代码会依次执行任务 1 和任务 2，每个任务暂停 1 秒钟。虽然它们看起来是同时进行的，但实际上是并发执行。

#### 并行

并行是指多个 CPU 或核心同时处理多个任务，多个任务在同一时刻各自独立运行。

In [ ]:
import multiprocessing
import time

# Set the start method for multiprocessing
multiprocessing.set_start_method('fork', force=True)

def task1():
    print("任务 1 开始")
    time.sleep(1)
    print("任务 1 完成")

def task2():
    print("任务 2 开始")
    time.sleep(1)
    print("任务 2 完成")

if __name__ == "__main__":  # Ensure code runs only in the main process
    p1 = multiprocessing.Process(target=task1)
    p2 = multiprocessing.Process(target=task2)

    p1.start()
    p2.start()

    p1.join()
    p2.join()


这段代码会让任务 1 和任务 2 并行执行。每个任务分别由不同的 CPU 核心处理，任务会同时进行。

### 多进程

#### 什么是进程

进程是操作系统分配资源的基本单位，每个进程都有独立的内存空间。一个进程的崩溃通常不会影响其他进程。

#### 使用 `multiprocessing.Process` 创建进程

在 Python 中，`multiprocessing` 模块允许我们创建进程。可以使用 `Process` 类来启动一个新的进程。

In [ ]:
import time
import multiprocessing

def write_file():
    with open("test.txt", "a") as f:
        while True:
            f.write("hello world\n")
            f.flush()
            time.sleep(0.5)

def read_file():
    with open("test.txt", "r") as f:
        while True:
            time.sleep(0.1)
            print(f.read(1))

if __name__ == "__main__":
    p1 = multiprocessing.Process(target=write_file)
    p2 = multiprocessing.Process(target=read_file)
    p1.start()
    p2.start()
    p1.join()
    p2.join()

这段代码创建了两个进程：一个用于写文件，另一个用于读文件。注意，在 Windows 上需要使用 `if __name__ == "__main__":` 来避免进程创建问题。

#### 自定义 `Process` 子类创建进程

你可以通过继承 `Process` 类来创建自定义进程类。

In [ ]:
import os
import multiprocessing

class Worker(multiprocessing.Process):
    def run(self):
        print("进程id：", os.getpid(), "\t父进程id：", os.getppid())

if __name__ == "__main__":
    for i in range(5):
        p = Worker()
        p.start()
        p.join()

#### 进程池

当需要启动大量进程时，使用进程池更为高效。可以通过 `multiprocessing.Pool` 创建进程池。

In [ ]:
import os
import time
import multiprocessing

def func():
    for i in range(10):
        print(os.getpid(), i)
        time.sleep(0.5)

if __name__ == "__main__":
    process_num = 5
    pool = multiprocessing.Pool(process_num)
    for _ in range(process_num):
        pool.apply_async(func)
    pool.close()
    pool.join()
    print("end")

这段代码创建了一个进程池，并向池中提交了多个任务。每个任务会在池中的进程上执行。

#### 进程间通信

1. **进程间不共享全局变量**

进程间不共享内存空间，因此，修改一个进程中的全局变量不会影响其他进程。

In [ ]:
import os
import multiprocessing

def func(list1):
    for i in range(10):
        list1.append(i)
        print(os.getpid(), list1)

if __name__ == "__main__":
    list1 = []
    p1 = multiprocessing.Process(target=func, args=(list1,))
    p2 = multiprocessing.Process(target=func, args=(list1,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(os.getpid(), list1)

这段代码创建了两个进程，它们都修改 `list1`，但由于进程间不共享内存，修改的结果会有差异。

2. **使用 `Queue` 通信**

可以通过 `Queue` 实现进程间的通信。

In [ ]:
import time
import random
import multiprocessing

def func1(queue):
    while True:
        queue.put(random.randint(1, 50))
        time.sleep(random.random())

def func2(queue):
    while True:
        print("=" * queue.get())

if __name__ == "__main__":
    queue = multiprocessing.Queue()
    p1 = multiprocessing.Process(target=func1, args=(queue,))
    p2 = multiprocessing.Process(target=func2, args=(queue,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

这段代码创建了两个进程，一个向 `Queue` 中放入数据，另一个从 `Queue` 中获取数据并打印。

### 多线程

#### 什么是线程

线程是处理器调度和执行的基本单位。一个进程至少有一个线程，线程之间可以共享数据。多个线程并发执行时，可以提高程序的效率。

#### 使用 `threading.Thread` 创建线程

Python 的 `threading` 模块提供了创建线程的功能。

In [ ]:
import time
import threading

def func():
    flag = 0
    while True:
        print(threading.current_thread().name, f"{flag}" * 5)
        flag = flag ^ 1  # 替换0和1
        time.sleep(0.5)

if __name__ == "__main__":
    t1 = threading.Thread(target=func, name="线程1")
    t2 = threading.Thread(target=func, name="线程2")
    t1.start()
    t2.start()

这段代码创建了两个线程，两个线程交替打印 `00000` 和 `11111`。

#### 自定义 `Thread` 子类创建线程

可以通过继承 `Thread` 类来创建自定义线程类。

In [ ]:
import time
import threading

class Worker(threading.Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        flag = 0
        while True:
            print(f"\r{self.name}:{str(flag)*5}", end="")
            flag = flag ^ 1  # 替换0和1
            time.sleep(0.2)

if __name__ == "__main__":
    t1 = Worker("线程1")
    t2 = Worker("线程2")
    t1.start()
    t2.start()

这段代码通过自定义 `Worker` 类创建了两个线程，线程交替打印 0 和 1。

#### 线程池

使用 `ThreadPoolExecutor` 创建线程池，并轻松管理任务。

In [ ]:
import concurrent.futures

def func(tname):
    global word
    for i, char in enumerate(word):
        word[i] = chr(ord(char) ^ 1)
        print(f"{tname}: {word}\n", end="")
    return word

if __name__ == "__main__":
    word = list("idmmn!vnsme")
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future1 = executor.submit(func, "线程1")
        future2 = executor.submit(func, "线程2")
        future3 = executor.submit(func, "线程3")
        word = future1.result()
        word = future2.result()
        word = future3.result()
    print("".join(word))  # hello world

这段代码创建了一个线程池，提交了多个任务，每个任务将字符列表与 1 进行异或运算。

#### 互斥锁

使用互斥锁确保线程安全，避免多个线程同时修改共享数据导致冲突。

In [ ]:
import time
import threading

def func():
    global g_num
    for _ in range(10):
        lock.acquire()  # 获取锁
        tmp = g_num + 1
        time.sleep(0.01)
        g_num = tmp
        lock.release()  # 释放锁
        print(f"{threading.current_thread().name}: {g_num}\n", end="")

if __name__ == "__main__":
    g_num = 0
    lock = threading.Lock()  # 创建锁
    threads = [threading.Thread(target=func, name=f"线程{i}") for i in range(3)]
    [t.start() for t in threads]
    [t.join() for t in threads]
    print(g_num)  # 30

这段代码使用互斥锁保证每个线程在修改 `g_num` 时不会发生冲突。

#### 14.3.6 GIL

Python 的 GIL（全局解释器锁）限制了多线程的并发性能，只有一个线程可以在同一时间执行 Python 字节码。对于 I/O 密集型任务，GIL 的影响较小，但对于 CPU 密集型任务，它会成为性能瓶颈。

### 14.4 进程和线程对比

#### 14.4.1 区别

1. **资源分配**

   * 进程拥有独立的内存空间和系统资源。每个进程有自己的代码段、数据段和堆栈等。
   * 线程共享所属进程的内存空间和资源，线程间可以直接访问共享的内存。

2. **开销**

   * 创建进程需要分配独立的内存、打开文件等系统资源，开销较大。
   * 创建线程只需在所属进程的内存空间内进行少量资源分配，开销较小。

3. **并发性**

   * 在多核心 CPU 环境下，进程和线程都可以异步执行，但进程之间的异步是“真正的”异步（每个进程在不同核心上并行执行）。
   * 线程的异步执行，特别是在单核心 CPU 上，是通过时间片轮转实现的“伪异步”。在多核心 CPU 上，线程可以实现并行，但由于 GIL 的存在，Python 中的多线程在 CPU 密集型任务中并不能充分利用多个核心。

4. **独立性**

   * 进程之间相互独立，一个进程的崩溃通常不会影响其他进程。
   * 同一进程中的线程之间相互影响，一个线程出现问题可能导致整个进程崩溃。

5. **通信**

   * 进程间通信（IPC）相对复杂，需要使用特殊的机制，如管道、消息队列、共享内存等。
   * 线程间通信比较简单，因为它们共享内存，可以直接访问共享变量。

#### 14.4.2 使用场景

1. **适合使用多线程的情况**

   * **I/O 密集型任务**：如网络请求、文件读写等。线程共享内存，切换开销小，在等待 I/O 操作完成的时间内可以切换到其他线程执行，提高整体效率。例如，程序需要同时从多个网站下载数据，使用多线程可以在等待网络响应时执行其他下载任务。
   * **对资源共享要求高**：线程间共享内存，方便数据共享和通信。例如，在一个图形界面程序中，多个线程需要共享界面数据并进行实时更新。

2. **适合使用多进程的情况**

   * **CPU 密集型任务**：多进程可以利用多核心 CPU 实现真正的并行计算，充分发挥硬件性能。例如，进行复杂的科学计算、数据处理等任务，每个进程在不同核心上独立计算，从而提高计算速度。
   * **需要隔离的任务**：进程相互独立，一个进程崩溃不会影响其他进程。对于一些可能出现异常或不稳定的任务，使用多进程可以保证系统的稳定性。例如，运行多个独立的服务，每个服务作为一个进程，避免一个服务出错影响其他服务。

### 总结

* **多线程**适用于 I/O 密集型任务和需要频繁共享数据的情况，线程间的切换开销较小，但由于 GIL，无法充分利用多个 CPU 核心。
* **多进程**适用于 CPU 密集型任务，可以在多核 CPU 上实现真正的并行计算。进程之间独立，互不干扰，适用于对任务进行隔离的场景。
